<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [7]:
# Write your code here
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup as BS
import re
import numpy as np
import pandas as pd

def get_tablelist(soup):
    reviews = soup.find_all("div", {"class": "lister-item-content"})

    review_data = []

    for review in reviews:

        try:
            review_content = review.find("div", {"class": "text show-more__control"}).text.strip()
        except:
            review_content = review.find("div", {"class": "text show-more__control clickable"}).text.strip()

        review_data.append(review_content)
    return review_data

b_url = 'https://www.imdb.com/'
url = 'https://www.imdb.com/title/tt9376612/reviews/?ref_=tt_ql_urv'
res = requests.get(url)
soup = BS(res.text, 'html.parser')
res.encoding = 'utf-8'
allTablelist=[]
tablelist = get_tablelist(soup)
for i in tablelist:
  allTablelist.append(i)

load_more = soup.select(".load-more-data")
flag = True
if len(load_more):
    ajaxurl = load_more[0]['data-ajaxurl']
    b_url = b_url + ajaxurl + "?ref_=undefined&paginationKey="
    try:
        key = load_more[0]['data-key']
    except KeyError:
        flag = False
else:
    flag = False

while flag:
    url = b_url + key
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BS(res.text, 'html.parser')
    tablelist2 = get_tablelist(soup)
    if len(allTablelist) == 100:
      break
    for i in tablelist2:
        allTablelist.append(i)
    load_more = soup.select(".load-more-data")
    if len(load_more):
        key = load_more[0]['data-key']
    else:
        flag = False

data = np.array(allTablelist)
data = pd.DataFrame(data, columns=['review_content'])
data.shape
data.to_csv('Reviews.csv')
data






,review_content
0,I got to admit I wasn't particularly blown awa...
1,The characters are a bit incoherent and straig...
2,RATED 8/10\nLanguage: English\nSource: Hotstar...
3,Everything comes in it beautiful science ficti...
4,The story was riddled with holes and plot issu...
...,...
95,"I loved the cast, music, plot, everything. I e..."
96,This was fantastic. Saw it in theaters then ag...
97,The buzz was high on this film but I kept my e...
98,The Action scenes were on par with the crouchi...


In [8]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
d = pd.read_csv('Reviews.csv', low_memory=False)
word_vectorizer = CountVectorizer(ngram_range=(3,3), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(data['review_content'].values.astype('U')) 
frequencies = sum(sparse_matrix).toarray()[0]
df = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['Frequency'])
df

,Frequency
05 2021 ______________________________________________________katy,1
10 from me,2
10 it was,1
10 language english,1
10 minutes don,1
...,...
zhang as his,1
zhang by their,1
zhang fala chen,1
zhang who disney,1


In [16]:
import csv
with open('Reviews.csv') as f:
    reader = csv.reader(f)
    next(reader, None)
    review_content = [row[1] for row in reader]
with open('review_content.txt', mode="w") as outfile:
    for s in review_content:
        outfile.write("%s\n" % s)
def bigramEstimation(file):
    lst = []
    unigrams = {}
    bigrams = {} 
    text = open(file, 'r').read()
    lst = text.strip().split()
    del text 
    for l in lst:
        if not l in unigrams:
            unigrams[l] = 1
        else:
            unigrams[l] += 1
    for i in range(len(lst) - 1):
        temp = (lst[i], lst[i+1])
        if not temp in bigrams:
            bigrams[temp] = 1
        else:
            bigrams[temp]+=1
    print('Generated ', len(bigrams), ' bigrams')
    total_corpus = sum(unigrams.values())
    for k,v in bigrams.items():
        first_word = k[0]
        first_word_count = unigrams[first_word]
        bi_prob = bigrams[k] / unigrams[first_word]
        if(v == 2):
         print(k[0] ,k[1],v ,bi_prob)

In [17]:
import spacy
import csv
from tabulate import tabulate


nlp = spacy.load("en_core_web_sm")
f = open("review_content.txt", "r")
doc = nlp(f.read())
noun_phrases = []
for np in doc.noun_chunks:
  noun_phrases.append(np.text)
print(noun_phrases)
dfn = pd.DataFrame(noun_phrases, columns = ['noun_phrases'])
word_vectorizer = CountVectorizer(ngram_range=(2,7), analyzer='word')
sparse_matrix = word_vectorizer.fit_transform(dfn['noun_phrases'].values.astype('U')) 
frequencies = sum(sparse_matrix).toarray()[0]
dff = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['Frequency'])
dff['Probability'] = dff['Frequency'] / dff['Frequency'].max()
result = dff.astype(object).transpose() 
result

['I', 'I', 'anything', 'the movie', 'I', 'I', 'it', 'me', 'it', 'a spectacle', 'The action', 'The locations', 'nice, costume design', 'sound design', 'it', 'a soft release', 'theaters', 'a follow-up', 'what', 'Endgame', 'The biggest problem', 'me', 'the underwritten lead', 'personality', 'reasons', 'the story', 'charm', 'personality', 'good dialogue', 'The next issue', 'the humor', 'which', 'me', 'The supporting lead', 'Awkwafina', 'her normal "loud" self', 'that', 'she', 'her mouth', 'Her role', 'I', 'myself', 'she', 'it', 'She', 'another forgettable character', 'that', 'more importance', 'the role', 'the third act', 'a balancing problem', 'stakes', 'You', 'tension', 'weight', 'any action', 'a lot', 'superficial', 'I', 'it', 'they', 'the end', 'this', 'a very safe plot', 'a way', 'china', 'the Mandarin', 'that', 'no one', 'a Marvel movie', 'it', 'we', 'different aspects', 'the narrative', 'it', 'a comic book', 'It', 'that', 'The characters', 'their own characterizations', 'times', 'th

,10 language,10 minutes,10 movie,10 outta,10 rings,10 stars,10 year,10 year olds,10 years,1000 year,...,your dragon,your face,your friends,your long,your long lost,your long lost brother,your toes,yuen wah,zero shock,zero shock value
Frequency,1,2,1,1,1,1,1,1,2,1,...,1,1,1,1,1,1,1,2,1,1
Probability,0.014085,0.028169,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.028169,0.014085,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.028169,0.014085,0.014085


# **Question 2: Undersand TF-IDF and Document representation**

(20 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [18]:
# Write your code here
df = pd.read_csv("Reviews.csv")
tf2 = df.dropna()
tf1 = (tf2['review_content'].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index())
tf1.columns = ['Word','Term_Frequency']
tf1






,Word,Term_Frequency
0,to,285.0
1,a,315.0
2,the,689.0
3,is,221.0
4,it,148.0
...,...,...
3779,loop,1.0
3780,..oh,1.0
3781,..overall,1.0
3782,boy,1.0


In [31]:
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
doc_set = tf2['review_content'].values.tolist()
query_set = "MCU aside, this movie stands alone as an epic masterpiece on its own! The fight scenes were visually stunning, great acting, great story highly reccomend!"
query_set = [query_set]
stopWords = stopwords.words('english')
vectorizer = CountVectorizer(stop_words = stopWords)
transformer = TfidfTransformer()
docVectorizerArray = vectorizer.fit_transform(doc_set).toarray()
queryVectorizerArray = vectorizer.transform(query_set).toarray()
cx = lambda a, b : np.inner(a, b)/(LA.norm(a)*LA.norm(b))
result = []
for vector in docVectorizerArray:
        for testV in queryVectorizerArray:
            cosine = cx(vector, testV)
            result.append(cosine)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vinayvarma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
n = tf2.filter(['Unnamed','review_content'], axis=1)
se = pd.Series(result)
n['Cos-similarity'] = se.values
n.drop(n.loc[n['Cos-similarity']==0].index, inplace=True)
n["Rank"] = n["Cos-similarity"].rank().astype(int)
n.sort_values("Cos-similarity", inplace = True) 
n

,review_content,Cos-similarity,Rank
22,I liked the first half of this film and I thou...,0.020064,1
50,Shang-Chi (Simu Liu) discovers that his all-po...,0.023415,2
61,Solid pop corn film. Although the director tri...,0.028172,3
28,Ben Kingsley won this movie. And it was not pa...,0.030429,4
72,Shang-Chi and the Legend of the Ten Rings (202...,0.032686,5
...,...,...,...
74,"""Shang-Chi and the Legend of the Ten Rings"" is...",0.244339,81
64,Best movie of the year. I waited for that kind...,0.251976,82
51,I love all the movies from them but Shang-Chi ...,0.264906,83
9,The best marvel movie after Endgame. The story...,0.326599,84


# **Question 3: Create your own word embedding model**

(20 points). Use the data you collected for assignment two to build a word embedding model: 

(1) Train a 300-dimension word embedding (it can be word2vec, glove, ulmfit, bert, or others).

(2) Visualize the word embedding model you created.

Reference: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

Reference: https://jaketae.github.io/study/word2vec/

In [25]:
from selenium import webdriver 
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [26]:
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
import pandas as pd
import time

driver = webdriver.Chrome('chromedriver',options=options)
link = 'https://www.imdb.com/title/tt9376612/reviews/?ref_=tt_ql_urv'
title_array = []
review_array = []
driver.get(link)
for num in range(4):
  driver.find_element(By.CLASS_NAME,"ipl-load-more__button").click()
  time.sleep(5)
  listOfTitle = driver.find_elements(By.CLASS_NAME, "title")
  listOfReviews = driver.find_elements(By.CLASS_NAME, "text")
for ele, sub_ele in zip(listOfTitle, listOfReviews):
      title_array.append((ele.text).replace('\n',''))
      review_array.append(sub_ele.text)
df = pd.DataFrame(list(zip(title_array, review_array)), columns =['Title', 'Review'])
print("Length of data frame is {0}".format(len(df)))
df

Length of data frame is 125


,Title,Review
0,Sort of disappointed..,I got to admit I wasn't particularly blown awa...
1,Questionable script (and Awkwafina is just the...,The characters are a bit incoherent and straig...
2,Worth a watch with family,RATED 8/10 Language: English Source: Hotstar R...
3,"beautiful science fiction, fantasy and action ...",Everything comes in it beautiful science ficti...
4,This could've been great if it wasn't for the ...,The story was riddled with holes and plot issu...
...,...,...
120,something unique and fresh,After 10 years of almost every movie being.arm...
121,When will Disney learn?,I heard great things on the internet about thi...
122,Marvel with a twist,
123,Another ruined opportunity.,"The whole thing is like a BMW commercial, shot..."


# **Question 4: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [33]:
# The GitHub link of your final csv file



# Link: 

#https://github.com/sathyak1/karra_INFO5731_Fall2021/blob/main/reviews.csv

